In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install beautifulsoup4 
!pip install requests

In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
base_url = 'https://2ip.ru/isp-rating/ru/%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D0%B0/'
response = requests.get(base_url)
base_soup = BeautifulSoup(response.text, "html.parser")

pages_count = int(base_soup.find_all('li', class_='pagination__item')[-2].text.strip())

In [ ]:
url_part1 = 'https://2ip.ru/isp-rating/ru/%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D0%B0/?pageId='
url_part2 = '&orderBy=ratingPlace&itemPerPage=30&direction=1'
response = requests.get(base_url)

In [ ]:
names = []
positions = []
rates = []
urls = []


for i in range(1, pages_count + 1):
    url = url_part1 + str(i) + url_part2
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    provs = soup.find_all('tr', class_='top')

    for p in provs:
        try:
            name = p.find('td', class_='name').find('a').text
        except:
            name = p.find('td', class_='name').text
        position = int(p.find('td', class_='number').text)
        rate = float(p.find_all('td')[2].text)

        r_count = p.find_all('td')[3].find_all('a')[0].text

        if r_count == '0':
            continue
        
        url_ = p.find_all('td')[3].find_all('a')[0].get('href')

        names.append(name.strip())
        positions.append(position)
        rates.append(rate)
        urls.append(url_)

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd

In [ ]:
from datetime import datetime

In [ ]:
# Парсинг отзывов по провайдерам
providers = []
revs = []

_url1 = '/?pageId='
_url2 = '&orderBy=id&itemPerPage=10'

for i in range(len(urls)):
    resp = requests.get(urls[i])
    soup = BeautifulSoup(resp.text, "html.parser")
    pages = soup.find_all('li', class_='pagination__item')

    if len(pages) > 1:
        p_count = int(pages[-2].text.strip())
    else:
        p_count = 1

    if p_count > 1000:
        continue
    
    for j in range(1, p_count + 1):
        resp = requests.get(urls[i] + _url1 + str(j) + _url2)
        rev_soup = BeautifulSoup(resp.text, "html.parser")

        reviews = rev_soup.find_all('div', class_='reviewItem__inner')
        for r in reviews:
            if names[i] != r.find('div', class_='reviewItem__provider').text.strip():
                break

            name = names[i]
            review = r.find('p', class_='reviewItem__txt').text.strip()

            providers.append(name)
            revs.append(review)

In [ ]:
reviews_data = {
    'Provider': providers,
    'Review': revs,
    # 'Date': dates
}

revs_df = pd.DataFrame(reviews_data)

In [ ]:
revs_df.to_csv('drive/MyDrive/labs/reviews.csv', index=False)

In [ ]:
for name in names:
    if not name in providers:
        i = names.index(name)
        names.remove(names[i])
        positions.remove(positions[i])
        rates.remove(rates[i])

data = {
    'Provider': names,
    'Position': positions,
    'Rating': rates
}

prov_df = pd.DataFrame(data)

In [ ]:
prov_df.to_csv('drive/MyDrive/labs/providers.csv', index=False)